In [1]:
%matplotlib inline

%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np

import xarray as xr

import regionmask

import pandas as pd

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import cartopy.feature as cfeat
import matplotlib.patches as mpatches
import datetime

import geopandas

import feather

import glob

import calendar

import sys
sys.path.append('../')
import utils
import plotting

np.seterr(invalid='ignore'); # disable a warning from matplotlib and cartopy


Bad key "text.kerning_factor" on line 4 in
/home/emfreese/anaconda3/envs/conda_env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


# GEOS Chem Runs

In [2]:
data_path = '/net/fs03/d1/emfreese/GCrundirs/nuclearproj/'

egrid_path = 'NA_egrid_2016'
NEI2016_path = 'NA_NEI_2016'
NEI2011_path = 'NA_oldNEI_2016'
normal_model_path = 'NA_normal_2016'
nonuc_model_path = 'NA_nonuc_2016'

output_path = '/merra2_05x0625_tropchem_na/OutputDir/'

speciesconc_output = 'GEOSChem.SpeciesConc.2016*.nc4'
aerosol_output = 'GEOSChem.AerosolMass.2016*.nc4'


In [3]:
ds_egrid = utils.import_GC_runs_general(data_path+egrid_path+output_path, speciesconc_output, aerosol_output, 'egrid')
ds_NEI2016 = utils.import_GC_runs_general(data_path+NEI2016_path+output_path, speciesconc_output, aerosol_output, 'NEI2016')
ds_NEI2011 = utils.import_GC_runs_general(data_path+NEI2011_path+output_path, speciesconc_output, aerosol_output, 'NEI2011')
ds_normal_model = utils.import_GC_runs_general(data_path+normal_model_path+output_path, speciesconc_output, aerosol_output, 'normal_model')
ds_nonuc_model = utils.import_GC_runs_general(data_path+nonuc_model_path+output_path, speciesconc_output, aerosol_output, 'nonuc_model')


In [4]:
datasets = [
    ds_egrid,
    ds_NEI2016,
    ds_NEI2011,
    ds_normal_model,
    ds_nonuc_model
]

In [5]:
#combine our datasetts
poll_ds = utils.combine_and_convert_ds(utils.gas_species_list, utils.aerosol_species_dict.keys(), 
                                       datasets, ['egrid', 'NEI2016', 'NEI2011', 'normal_model','nonuc_model'],['egrid','NEI2016','NEI2011','nonuc_model'], 
                                       'normal_model', '5x_models')




In [21]:
#save our dataset
xr.Dataset.to_zarr(poll_ds, './data/GC_output.zarr', mode = 'w') #save the dataset 

/home/emfreese/anaconda3/envs/conda_env/lib/python3.7/site-packages/dask/core.py:119: RuntimeWarning: overflow encountered in multiply
  return func(*args2)
/home/emfreese/anaconda3/envs/conda_env/lib/python3.7/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in true_divide
  return func(*args2)
/home/emfreese/anaconda3/envs/conda_env/lib/python3.7/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in subtract
  return func(*args2)
/home/emfreese/anaconda3/envs/conda_env/lib/python3.7/site-packages/dask/core.py:119: RuntimeWarning: overflow encountered in true_divide
  return func(*args2)
/home/emfreese/anaconda3/envs/conda_env/lib/python3.7/site-packages/dask/core.py:119: RuntimeWarning: divide by zero encountered in true_divide
  return func(*args2)


# US-EGO Generation and Emissions Data

#### Definition of Plant type (in 2016 EPA egrid TSD section 5)
The fuel category for the primary fuel of the plant. This field is “COAL” if the plant’s primary fuel is derived from coal (fuel type = BIT, COG, LIG, RC, SGC, SUB, WC), “OIL” if it is derived from oil (DFO, JF, KER, PC, RFO, WO), “GAS” if it is derived from gas (BU, NG, PG), “OFSL” if it is another fossil fuel (BFG, OG, TDF), “NUCLEAR” if it is derived from nuclear (NUC), “HYDRO” if it is derived from hydro power (WAT), “SOLAR” if it is derived from solar power, (SUN), “WIND” if it is derived from wind power (WND), “GEOTHERMAL” if it is derived from geothermal power (GEO), “OTHF” if it is derived from waste heat/unknown/purchased (MWH, OTH, PRG, PUR, WH), and “BIOMASS” if it is derived from biomass sources (AB, BLQ, LFG, MSW, OBG, OBL, OBS, SLW, WDL, WDS).

In [30]:
##### import files and change hydro #####

###get ORIS, plant type, and Region Name from our modified generation file
oris_nonuc_df = pd.read_csv('../optimization_model/good_model_inputs/inputs_gen_no-nuclear_all-generators_20k-new_name.csv',usecols=[1,2,4,5,8,22,23,24,25])
oris_normal_df = pd.read_csv('../optimization_model/good_model_inputs/inputs_gen_normal.csv',usecols=[1,2,4,5,8,22,23,24,25])
oris_nonuc_df.loc[oris_nonuc_df['FuelType'] == 'Pumps', 'FuelType'] = 'Hydro' #change pumps to hydro label
oris_normal_df.loc[oris_normal_df['FuelType'] == 'Pumps', 'FuelType'] = 'Hydro' #change pumps to hydro label

###import egrid generation data and modify the index to be ORISCode
egrid_df_raw = pd.read_excel('../raw_data/egrid2016_data.xlsx',sheet_name='GEN16', usecols='D, J, L')
egrid = egrid_df_raw.drop(egrid_df_raw.index[0]).rename(columns={'DOE/EIA ORIS plant or facility code':'ORISCode'})
#change those with no capacity to 1 (so that our capacity factor calculation doesn't fail)
egrid.loc[egrid['Generator nameplate capacity (MW)'] == 0, 'Generator nameplate capacity (MW)'] = 1
#calculate the capacity factor of generation in egrid
egrid['egrid_capafactor'] = egrid['Generator annual net generation (MWh)'] / (8760 * egrid['Generator nameplate capacity (MW)']) 

###import the generation data from our optimization
gen_normal_df=feather.read_dataframe('../optimization_model/outputs/gen_normal.feather')

###import the generation data from our optimization
gen_nonuc_df=feather.read_dataframe('../optimization_model/outputs/gen_no-nuclear_modified-all-generators.feather')

### import the plant type from our egrid dataset
planttype_df_raw=pd.read_excel('../raw_data/egrid2016_data.xlsx',sheet_name='PLNT16', usecols='D, W')
planttype_df=planttype_df_raw.drop(egrid_df_raw.index[0]).rename(columns={'DOE/EIA ORIS plant or facility code':'ORISCode', 
                                                                       'Plant primary coal/oil/gas/ other fossil fuel category':'planttype'})



In [31]:
##### change index and convert to xarray #####

#group by ORIS code and make a date and ORIS code multi index
gen_nonuc_df = pd.concat([gen_nonuc_df,oris_nonuc_df['ORISCode']], axis = 1).groupby(['ORISCode']).sum()
gen_normal_df = pd.concat([gen_normal_df,oris_normal_df['ORISCode']], axis = 1).groupby(['ORISCode']).sum()
gen_normal_df = gen_normal_df.stack()
gen_nonuc_df = gen_nonuc_df.stack()
gen_nonuc_df.index.names = (['ORISCode','date'])
gen_normal_df.index.names = (['ORISCode','date'])

egrid = egrid.groupby('ORISCode').sum().drop(columns = 'egrid_capafactor') #drop capacity factor because that isn't the sum
 
planttype=planttype_df.set_index(['ORISCode'])
planttype_sort=planttype.sort_values(by="ORISCode")

egrid_planttype_df = pd.concat((egrid,planttype_sort),axis=1)

#convert to xarray datasets
oris_nonuc_ds = oris_nonuc_df.to_xarray()
oris_normal_ds = oris_normal_df.to_xarray()
egrid_planttype_ds = egrid_planttype_df.to_xarray()
gen_normal_ds = gen_normal_df.to_xarray()
gen_nonuc_ds = gen_nonuc_df.to_xarray()

In [32]:
#### combine our egrid and generation info for no nuclear and normal models ####
nonuc_gmodel_egrid_ds = utils.combine_egrid_generation(oris_nonuc_ds, gen_nonuc_ds, egrid_planttype_ds)
normal_gmodel_egrid_ds = utils.combine_egrid_generation(oris_normal_ds, gen_normal_ds, egrid_planttype_ds)
gmodel_egrid_raw_ds = xr.concat([nonuc_gmodel_egrid_ds, normal_gmodel_egrid_ds], pd.Index(['nonuc_model','normal_model'], name='model_name'))

/home/emfreese/anaconda3/envs/conda_env/lib/python3.7/site-packages/xarray/core/nanops.py:142: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


In [33]:
#### fix datetime and add dec 31st at the 23rd hour ####

###create a dict for dec 31st 23rd hour (just copy the dec 31 22nd hour data over)
dec23_ds = gmodel_egrid_raw_ds.copy().isel(date = [-1])
#change date to datetime for the 23rd hour
dec23_ds['date'] = [datetime.datetime(2017,12,31,23,0)]

###change all dates to datetime
#make a list of dates for the year
base = datetime.datetime(2017, 1, 1) #base date
date_list = [base + datetime.timedelta(hours=x) for x in range(8759)] #loop through all hours but the final one (we will add it in above)
date_list

gmodel_egrid_raw_ds['date'] = date_list
gmodel_egrid_ds = xr.merge([gmodel_egrid_raw_ds, dec23_ds])

#### Note:

To convert our generation in MWh to get the emissions in kg/sec, we use the following (and making sure our emissions rates are in kg/sec)

$\text{Mwh}/3600sec -> (\text{Mw}/s) * kg/\text{Mw} -> kg/s$


In [34]:
###### add NOx, SO2, CO2, CH4 emissions to gmodel_egrid dataset ######

no_mult = 0.8544304 # NO/NOx as estimated from NEI2011 inventory
no2_mult = 1 - 0.8544304 # NO2/NOx as estimated from NEI2011 inventory

### process our emissions by multiplying generation* emissions factors
gmodel_egrid_ds['model_NO_rate']  = no_mult * gmodel_egrid_ds['modelgeneration']/3600 * gmodel_egrid_ds['PLNOXRTA']
gmodel_egrid_ds['model_NO2_rate']  = no2_mult * gmodel_egrid_ds['modelgeneration']/3600 * gmodel_egrid_ds['PLNOXRTA']
gmodel_egrid_ds['model_SO2_rate']  =  gmodel_egrid_ds['modelgeneration']/3600 * gmodel_egrid_ds['PLSO2RTA']
gmodel_egrid_ds['model_CO2_rate']  =  gmodel_egrid_ds['modelgeneration']/3600 * gmodel_egrid_ds['PLCO2RTA']
gmodel_egrid_ds['model_CH4_rate']  =  gmodel_egrid_ds['modelgeneration']/3600 * gmodel_egrid_ds['PLCH4RTA']

gmodel_egrid_ds['egrid_annual_NO_rate']  = no_mult * gmodel_egrid_ds['annual_egridgeneration']/3600 * gmodel_egrid_ds['PLNOXRTA']
gmodel_egrid_ds['egrid_annual_NO2_rate']  = no2_mult * gmodel_egrid_ds['annual_egridgeneration']/3600 * gmodel_egrid_ds['PLNOXRTA']
gmodel_egrid_ds['egrid_annual_SO2_rate']  =  gmodel_egrid_ds['annual_egridgeneration']/3600 * gmodel_egrid_ds['PLSO2RTA']
gmodel_egrid_ds['egrid_annual_CO2_rate']  =  gmodel_egrid_ds['annual_egridgeneration']/3600 * gmodel_egrid_ds['PLCO2RTA']
gmodel_egrid_ds['egrid_annual_CH4_rate']  =  gmodel_egrid_ds['annual_egridgeneration']/3600 * gmodel_egrid_ds['PLCH4RTA']

##### annual generation in original dataset ######
gmodel_egrid_ds['annual_modelgeneration'] = gmodel_egrid_ds['modelgeneration'].groupby('ORISCode').sum(dim = 'date')

/home/emfreese/anaconda3/envs/conda_env/lib/python3.7/site-packages/xarray/core/common.py:664: FutureWarning: This DataArray contains multi-dimensional coordinates. In the future, the dimension order of these coordinates will be restored as well unless you specify restore_coord_dims=False.
  self, group, squeeze=squeeze, restore_coord_dims=restore_coord_dims


In [35]:
####### create new datasets for region and fuel type grouped data, add annual generation to normal ds ######

###list of pollutants
pollutants = ['NO','NO2','SO2','CO2','CH4']

def create_grouped_ds(grouper, model):
    grouped_ds = gmodel_egrid_ds.copy().sel(model_name = model).groupby(grouper).sum()
    grouped_ds = grouped_ds.drop(['model_capafactor','PLCH4RTA','PLCO2RTA','PLNOXRTA','PLSO2RTA','PLN2ORTA'])
    return(grouped_ds)

###create datasets of grouped fuel type and region name, combine
nonuc_fueltype_grouped_ds = create_grouped_ds('fueltype', 'nonuc_model')
normal_fueltype_grouped_ds = create_grouped_ds('fueltype', 'normal_model')

nonuc_regionname_grouped_ds = create_grouped_ds('regionname', 'nonuc_model')
normal_regionname_grouped_ds = create_grouped_ds('regionname', 'normal_model')

fueltype_grouped_ds = xr.concat([nonuc_fueltype_grouped_ds, normal_fueltype_grouped_ds], pd.Index(['nonuc_model','normal_model'], name='model_name'))
regionname_grouped_ds = xr.concat([nonuc_regionname_grouped_ds, normal_regionname_grouped_ds], pd.Index(['nonuc_model','normal_model'], name='model_name'))

In [37]:
#add annual emissions of each pollutant
for pollutant in pollutants:
    regionname_grouped_ds[f'model_annual_{pollutant}_conc'] = (regionname_grouped_ds[f'model_{pollutant}_rate']*3600).sum(dim = 'date')
    fueltype_grouped_ds[f'model_annual_{pollutant}_conc'] = (fueltype_grouped_ds[f'model_{pollutant}_rate']*3600).sum(dim = 'date')
    gmodel_egrid_ds[f'model_annual_{pollutant}_conc'] = (gmodel_egrid_ds[f'model_{pollutant}_rate']*3600).sum(dim = 'date')

    regionname_grouped_ds[f'egrid_annual_{pollutant}_conc'] = (regionname_grouped_ds[f'egrid_annual_{pollutant}_rate']*3600)
    fueltype_grouped_ds[f'egrid_annual_{pollutant}_conc'] = (fueltype_grouped_ds[f'egrid_annual_{pollutant}_rate']*3600)
    gmodel_egrid_ds[f'egrid_annual_{pollutant}_conc'] = (gmodel_egrid_ds[f'egrid_annual_{pollutant}_rate']*3600)

#add ratio of difference in emissions normal-nonuc/ difference in generation normal-nonuc to the nonuc dataset
for pollutant in pollutants:
    fueltype_grouped_ds.sel(model_name = 'nonuc_model')[f'normal-nonuc_{pollutant}-gen_ratio'] = (fueltype_grouped_ds.sel(model_name = 'normal_model')[f'model_annual_{pollutant}_conc']-
                                                                       fueltype_grouped_ds.sel(model_name = 'nonuc_model')[f'model_annual_{pollutant}_conc'])/(fueltype_grouped_ds.sel(model_name = 'normal_model')['annual_modelgeneration']-
                                                                                                                             fueltype_grouped_ds.sel(model_name = 'nonuc_model')['annual_modelgeneration'])
    regionname_grouped_ds.sel(model_name = 'nonuc_model')[f'normal-nonuc_{pollutant}-gen_ratio'] = (regionname_grouped_ds.sel(model_name = 'normal_model')[f'model_annual_{pollutant}_conc']-
                                                                         regionname_grouped_ds.sel(model_name = 'nonuc_model')[f'model_annual_{pollutant}_conc'])/(regionname_grouped_ds.sel(model_name = 'normal_model')['annual_modelgeneration']-
                                                                                                                                 regionname_grouped_ds.sel(model_name = 'nonuc_model')['annual_modelgeneration'])



In [38]:
#### add attributes ####
for pollutant in pollutants:
    if pollutant == 'NO2' or 'NO':
                gmodel_egrid_ds[f'PLNOXRTA'].attrs['units'] = 'kg/Mwh'
    else:
        gmodel_egrid_ds[f'PL{pollutant}RTA'].attrs['units'] = 'kg/Mwh'
    for idx, ds in enumerate([regionname_grouped_ds, fueltype_grouped_ds, gmodel_egrid_ds]):
        ds[f'model_{pollutant}_rate'].attrs['units'] = 'kg/s'
        ds[f'modelgeneration'].attrs['units'] = 'Mwh'
        for typem in ['egrid','model']:
            ds[f'annual_{typem}generation'].attrs['units'] = 'Mwh'
            ds[f'{typem}_annual_{pollutant}_conc'].attrs['units'] = 'kg'
        ds[f'egrid_annual_{pollutant}_rate'].attrs['units'] = 'kg/s'
        ds.attrs['group'] = ['Region','Fuel Type','All'][idx]
        
        

In [39]:
##### save datasets #####

### Final datasets to save are: gmodel_egrid, fueltype_grouped_ds, regionname_grouped_ds
regionname_grouped_ds.to_netcdf(f'./data/regionname_grouped_emissions_ds.nc', 'w')
fueltype_grouped_ds.to_netcdf(f'./data/fueltype_grouped_emissions_ds.nc', 'w')
gmodel_egrid_ds.to_netcdf(f'./data/gmodel_egrid_emissions_ds.nc', 'w')

### datasets partway to save are:

oris_nonuc_ds.to_netcdf('./data/oris_nonuc_ds.nc', 'w')
oris_normal_ds.to_netcdf('./data/oris_normal_ds.nc', 'w')
egrid_planttype_ds.to_netcdf('./data/egrid_ds.nc', 'w')
gen_normal_ds.to_netcdf('./data/gen_normal_ds.nc', 'w')
gen_nonuc_ds.to_netcdf('./data/gen_nonuc_ds.nc', 'w')

# MERRA2 RH and T

In [40]:
##### import MERRA2 datasets for RH and T #####

#import 
T_ds = xr.open_mfdataset('../../MERRA2/2016/GEOS_0.5x0.625_NA/MERRA2/2016/*/MERRA2.2016*.A1.05x0625.NA.nc4', combine = 'by_coords')
RH_ds = xr.open_mfdataset('../../MERRA2/2016/GEOS_0.5x0.625_NA/MERRA2/2016/*/MERRA2.2016*.A3dyn.05x0625.NA.nc4', combine = 'by_coords')

#reduce datasets just to T and RH
T_ds = T_ds['TS']
RH_ds = RH_ds['RH']

In [41]:
##### Select T and RH in the bounds of our US lat and lon #####
T = {}
RH = {}
for region in utils.lat_lon_dict.keys():
    T[region] = {}
    T[region] = T_ds.sel(
        lon = slice(utils.lat_lon_dict[region][0], utils.lat_lon_dict[region][1]), 
        lat = slice(utils.lat_lon_dict[region][2],utils.lat_lon_dict[region][3])
    ).groupby('time.season').mean()
    RH[region] = {}
    RH[region] = RH_ds.sel(
        lon = slice(utils.lat_lon_dict[region][0], utils.lat_lon_dict[region][1]), 
        lat = slice(utils.lat_lon_dict[region][2],utils.lat_lon_dict[region][3])
    ).groupby('time.season').mean()

In [42]:
##### Calculate seasonal mean RH and T by region #####
T_seasonal_mean = {}
RH_seasonal_mean = {}
for region in utils.lat_lon_dict.keys():
    T_seasonal_mean[region] = {}
    RH_seasonal_mean[region] = {}
    for seasons in ['DJF','JJA','MAM','SON']:
        T_seasonal_mean[region][seasons] = T[region].sel(season = seasons).mean().values
        RH_seasonal_mean[region][seasons] = RH[region].sel(season = seasons).mean().values

In [43]:
##### convert to dataframes ##### 

T_df = pd.DataFrame.from_dict({(i,j): T_seasonal_mean[i][j]
                            for i in T_seasonal_mean.keys() 
                            for j in T_seasonal_mean[i].keys()},
                            orient='index', columns = ['T'])
T_df.index = pd.MultiIndex.from_tuples(T_df.index)

RH_df = pd.DataFrame.from_dict({(i,j): RH_seasonal_mean[i][j]
                            for i in RH_seasonal_mean.keys() 
                            for j in RH_seasonal_mean[i].keys()},
                            orient='index', columns = ['RH'])
RH_df.index = pd.MultiIndex.from_tuples(RH_df.index)



In [44]:
##### merge dataframes #####
RH_T_df = pd.merge(
    T_df, RH_df, 
    left_index=True, right_index=True)

In [45]:
##### save dataframe #####
pd.DataFrame.to_csv(RH_T_df, './data/RH_T.csv')

# Observational Data

## EPA Observational data
data is from https://aqs.epa.gov/aqsweb/airdata/download_files.html
for the year 2016
choosing O3, NO, SO2, PM25 (FEM/FRM)


In [2]:
EPA_obs_df = utils.import_and_edit_EPAobs('../../GEOS_CHEM/obs_data/daily*.csv')

## IMPROVE Observational data

mw_dict = {'ammNO3': 80.043,
           'ammSO4': 132.14} #g/mol 
g_ug = 1e6

for species in mw_dict.keys():
    ammon_df[f'{species}f:Value'] *= mw_dict[species] * g_ug

In [3]:
pm_df = utils.import_IMPROVE('../../GEOS_CHEM/obs_data/IMPROVE_2016_PM.txt', 'PM25', 'MF')
s_df = utils.import_IMPROVE('../../GEOS_CHEM/obs_data/IMPROVE_2016_Sulfate.txt', 'SO4', 'SO4f')
n_df = utils.import_IMPROVE('../../GEOS_CHEM/obs_data/IMPROVE_2016_Nitrate.txt', 'NIT', 'NO3f')
ammon_df = utils.import_IMPROVE('../../GEOS_CHEM/obs_data/IMPROVE_2016_ammonia.txt', 'NH4', 'NH4f')
oc_df = utils.import_IMPROVE('../../GEOS_CHEM/obs_data/IMPROVE_2016_OC.txt', 'OC', 'ECf')

IMPROVE_df = pd.concat([pm_df, s_df, n_df,oc_df, ammon_df], axis = 0) #concatenate all dataframes and reset the index
IMPROVE_df['Date'] = pd.to_datetime(IMPROVE_df['Date']) #change to datetime
IMPROVE_df = IMPROVE_df.loc[IMPROVE_df['Arithmetic Mean'] >= 0] #get rid of -999 readings where there is no data
IMPROVE_df = IMPROVE_df.loc[(IMPROVE_df['Latitude'].between(24,50,inclusive = True)) & (IMPROVE_df['Longitude'].between(-130,-60,inclusive = True))]


## GC Data

In [4]:
### import GC dataset
poll_ds = xr.open_zarr('./data/GC_output.zarr')

## Lists/Dicts of Variables

In [5]:
#define Lat and Lon of the nested grid US
levels_dict = {'PM25':np.arange(0., 20., .5), 'SO2':np.arange(0., 5., .1), 
               'NO2':np.arange(0., 5., .1), 'NOx':np.arange(0., 5., .1), 'O3':np.arange(0., 70., 1.),
               'dif':np.arange(-.3, .31, .01), 'regional_dif':np.arange(-1.5, 1.51, .01)}

## Interpolate data to get GC run data at observational points

### EPA Interpolation

In [6]:
### interpolate data for EPA
interp_EPA_df = pd.DataFrame(columns=['Arithmetic Mean', 'Longitude', 'Latitude','model_name','species','date'])

for model in poll_ds['model_name'].values:
    for species in ['PM25', 'SO2', 'NO2', 'O3']:
        for month in pd.DatetimeIndex(EPA_obs_df.loc[EPA_obs_df.date.dt.month.isin(utils.month_string)]['date'].values).month.unique().values:

            #data selected for date
            data = poll_ds.sel(model_name = model)[f'{species}'].groupby('time.month').mean().sel(month = month)
            
            #new lat and lon in radians
            lats_new = EPA_obs_df.loc[(EPA_obs_df['species'] == species)]['Latitude'].unique()
            lons_new = EPA_obs_df.loc[(EPA_obs_df['species'] == species)]['Longitude'].unique()
            
            #interpolation function
            interp_data = []
            for idx in range(lats_new.size):
                interp_data.append(data.sel(lat=lats_new[idx], lon=lons_new[idx], method='nearest').values.item())
            tmp_df = pd.DataFrame({'Arithmetic Mean':interp_data, 'Longitude':lons_new, 'Latitude':lats_new, 'model_name': model, 'species': species, 'date': month})
            interp_EPA_df = interp_EPA_df.append(tmp_df, sort=False, ignore_index=True)

    

/home/emfreese/anaconda3/envs/conda_env/lib/python3.7/site-packages/numpy/core/_methods.py:36: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)
/home/emfreese/anaconda3/envs/conda_env/lib/python3.7/site-packages/numpy/core/_methods.py:36: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)
/home/emfreese/anaconda3/envs/conda_env/lib/python3.7/site-packages/numpy/core/_methods.py:36: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)
/home/emfreese/anaconda3/envs/conda_env/lib/python3.7/site-packages/numpy/core/_methods.py:36: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)
/home/emfreese/anaconda3/envs/conda_env/lib/python3.7/site-packages/numpy/core/_methods.py:36: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out,

In [7]:
for i in range(0, len(interp_EPA_df)):
    interp_EPA_df.loc[i,('date')] = datetime.datetime(2016,interp_EPA_df['date'][i],calendar.monthrange(2016,interp_EPA_df['date'][i])[1])

### IMPROVE Interpolation

In [8]:
### interpolate data for EPA
interp_IMPROVE_df = pd.DataFrame(columns=['Arithmetic Mean', 'Longitude', 'Latitude','model_name','species','date'])

for model in poll_ds['model_name'].values:
    for species in ['PM25','NIT','SO4','NH3','NH4','HNO3', 'TotalOC']:
        for month in pd.DatetimeIndex(IMPROVE_df.loc[IMPROVE_df.Date.dt.month.isin(utils.month_string)]['Date'].values).month.unique().values:

            #data selected for date
            data = poll_ds.sel(model_name = model)[f'{species}'].groupby('time.month').mean().sel(month = month)
            
            #new lat and lon in radians
            if species == 'NH3': #interpolating NH3 to get it for our ISORROPIA total NH4 and NH3
                lats_new = IMPROVE_df.loc[(IMPROVE_df['species'] == 'NH4')]['Latitude'].unique()
                lons_new = IMPROVE_df.loc[(IMPROVE_df['species'] == 'NH4')]['Longitude'].unique()
            if species == 'HNO3': #interpolating NH3 to get it for our ISORROPIA total HNO3 and NIT
                lats_new = IMPROVE_df.loc[(IMPROVE_df['species'] == 'NIT')]['Latitude'].unique()
                lons_new = IMPROVE_df.loc[(IMPROVE_df['species'] == 'NIT')]['Longitude'].unique()
            if species == 'TotalOC':
                lats_new = IMPROVE_df.loc[(IMPROVE_df['species'] == 'OC')]['Latitude'].unique()
                lons_new = IMPROVE_df.loc[(IMPROVE_df['species'] == 'OC')]['Longitude'].unique()
            else:
                lats_new = IMPROVE_df.loc[(IMPROVE_df['species'] == species)]['Latitude'].unique()
                lons_new = IMPROVE_df.loc[(IMPROVE_df['species'] == species)]['Longitude'].unique()
            #interpolation function
            interp_data = []
            for idx in range(lats_new.size):
                interp_data.append(data.sel(lat=lats_new[idx], lon=lons_new[idx], method='nearest').values.item())
            tmp_df = pd.DataFrame({'Arithmetic Mean':interp_data, 'Longitude':lons_new, 'Latitude':lats_new, 'model_name': model, 'species': species, 'date': month})
            interp_IMPROVE_df = interp_IMPROVE_df.append(tmp_df, sort=False, ignore_index=True)



In [9]:
for i in range(0, len(interp_IMPROVE_df)):
    interp_IMPROVE_df.loc[i,('date')] = datetime.datetime(2016,interp_IMPROVE_df['date'][i],calendar.monthrange(2016,interp_IMPROVE_df['date'][i])[1])

## Create a monthly observational dataframe

In [10]:
IMPROVE_df = IMPROVE_df.rename(columns = {'Date':'date'})

In [154]:
def create_monthly_obs_df(obs_df):
    
    #create the 'geometries' for each lat and lon
    gdf = geopandas.GeoDataFrame(
    obs_df, geometry=geopandas.points_from_xy(obs_df.Longitude, obs_df.Latitude))
    geometries = gdf['geometry'].apply(lambda x: x.wkt).values
    #add to the dataset
    obs_df['geometry'] = geometries
    obs_df.index = obs_df['date']

    #group by month and geometry
    monthly_obs_df = pd.DataFrame(columns = ['Arithmetic Mean','Latitude','Longitude', 'geometry','species', 'date'])
    geometry = geometries[0]
    for geometry in np.unique(np.unique(obs_df['geometry'])):
        for species in np.unique(obs_df['species'].values):
            lat = obs_df.loc[(obs_df['geometry'] == geometry) & (obs_df['species'] == species)].groupby(pd.Grouper(freq='M'))['Latitude'].first().values
            lon = obs_df.loc[(obs_df['geometry'] == geometry) & (obs_df['species'] == species)].groupby(pd.Grouper(freq='M'))['Longitude'].first().values
            data = obs_df.loc[(obs_df['geometry'] == geometry) & (obs_df['species'] == species)].groupby(pd.Grouper(freq='M'))['Arithmetic Mean'].mean()
            tmp_df = pd.DataFrame({'Arithmetic Mean': data.values, 'Latitude':lat, 'Longitude':lon, 
                                   'geometry':geometry, 'species': species, 'date': data.index})
            monthly_obs_df = monthly_obs_df.append(tmp_df, sort=False, ignore_index=True)
    return(monthly_obs_df)


In [155]:
monthly_IMPROVE_df = create_monthly_obs_df(IMPROVE_df)
monthly_EPA_df = create_monthly_obs_df(EPA_obs_df)

## Add Region Names to the dataframes

In [33]:
monthly_EPA_df

In [156]:
#add region to the dataframes based on lat_lon dictionary
for df in [IMPROVE_df, interp_IMPROVE_df, interp_EPA_df, EPA_obs_df, monthly_EPA_df, monthly_IMPROVE_df]:
    df['Region'] = 'a'
    for region in ['SE_lat_lon', 'NW_lat_lon', 'NE_lat_lon', 'MW_lat_lon', 'SW_lat_lon']:
        df.loc[(df['Longitude'].between(utils.lat_lon_dict[region][0], utils.lat_lon_dict[region][1], inclusive = True)) & 
            (df['Latitude'].between(utils.lat_lon_dict[region][2], utils.lat_lon_dict[region][3], inclusive = True)), 'Region'] = region

## Save the dataframes

In [157]:
interp_EPA_df.to_csv('./data/interp_EPA_df.csv', date_format='%Y%m%d', index=False)
EPA_obs_df.to_csv('./data/EPA_obs_df.csv', date_format='%Y%m%d', index=False)
monthly_EPA_df.to_csv('./data/EPA_monthly_obs_df.csv', date_format='%Y%m%d', index=False)
IMPROVE_df.to_csv('./data/IMPROVE_df.csv', date_format='%Y%m%d', index=False)
interp_IMPROVE_df.to_csv('./data/interp_IMPROVE_df.csv', date_format='%Y%m%d', index=False)
monthly_IMPROVE_df.to_csv('./data/IMPROVE_monthly_obs_df.csv', date_format='%Y%m%d', index=False)

# ISORROPIA Data

## Create Dataset

In [158]:
#import GC run data
poll_ds = xr.open_zarr('./data/GC_output.zarr')

#import RH and T dataframe
RH_T_df = pd.read_csv('./data/RH_T.csv', index_col=[0,1])

#subset our ds into just the species we need
species_list = ['SO4','NH3','HNO3','NIT','NH4']
isorropia_ds = poll_ds[species_list]

##### species in ISORROPIA 
SO4, NH3, NO3, Cl, Ca, K, Mg

In [159]:
#create a dataframe for our isorropia monthly mean values for no nuclear and normal cases, indexed by species, location, season

isorropia_dict = {}
for species in species_list:
    isorropia_dict[species]= {}
    for region in utils.lat_lon_dict.keys():
        isorropia_dict[species][region] = {}
        for season in ['DJF','MAM','JJA','SON']:
            data = isorropia_ds.sel(lon = slice(utils.lat_lon_dict[region][0], utils.lat_lon_dict[region][1]), 
                                    lat = slice(utils.lat_lon_dict[region][2],utils.lat_lon_dict[region][3])).groupby('time.season').mean(dim = ['lat','lon','time']).sel(season = season)[species]
            isorropia_dict[species][region][season] = data.values
            
isorropia_df = pd.DataFrame.from_dict({(i,j,k): isorropia_dict[i][j][k] 
                            for i in isorropia_dict.keys() 
                            for j in isorropia_dict[i].keys()
                            for k in isorropia_dict[i][j].keys()},
                            orient='index', columns = isorropia_ds['model_name'].values)

isorropia_df.index = pd.MultiIndex.from_tuples(isorropia_df.index, names = ['Species','Location', 'Season'])

In [160]:
#convert from mol/mol to mol/m3 by multiplying by P/T*R (mol/m3)
stp_p = 101325 #Pa = kg/m/s^2
R = 8.314 #J/K/mol
for species in species_list:
    for region in utils.lat_lon_dict.keys():
            for season in ['DJF','MAM','JJA','SON']:
                isorropia_df.loc[(species,region,season)] *= stp_p/R/RH_T_df.loc[(region,season)]['T']
                #isorropia_df.loc[(species,region,season)]['normal'] *= stp_p/R/RH_T_df.loc[(region,season)]['T']


In [161]:
#convert NIT, NH4, SO4 from ug/m3 to mol/m3
for species in utils.aerosol_species_dict.keys():
    for region in utils.lat_lon_dict.keys():
            for season in ['DJF','MAM','JJA','SON']:
                isorropia_df.loc[(species,region,season)] /= (utils.aerosol_species_dict[species]*1e6)#ug/m3/(#g/mol*#ug/g)

#convert NH3, HNO3 from ppbv to mol/m3
for species in ['NH3','HNO3']:
    for region in utils.lat_lon_dict.keys():
            for season in ['DJF','MAM','JJA','SON']:
                isorropia_df.loc[(species,region,season)] *= stp_p/R/RH_T_df.loc[(region,season)]['T']/1e9


In [162]:
#calculate the total Nitrate and ammonium by molarity
isorropia_df_TNO3 = isorropia_df.loc['HNO3'] + isorropia_df.loc['NIT'] 
isorropia_df_TNH3 = isorropia_df.loc['NH3'] + isorropia_df.loc['NH4'] 

## Run through ISORROPIA
### Only run once to initiate a new ISORROPIA output, otherwise, the output is already made

In [169]:
import os
os.chdir('/net/fs11/d0/emfreese/ISORROPIAIIStandalone')
for model in ['nonuc_model', 'normal_model']:
    for region in ['SE_lat_lon', 'NW_lat_lon', 'NE_lat_lon', 'MW_lat_lon', 'SW_lat_lon']:
            for season in ['DJF','JJA']:
                T_tmp = RH_T_df.loc[(region, season)]['T']
                RH_tmp = RH_T_df.loc[(region, season)]['RH']
                SO4_tmp = isorropia_df.loc['SO4',region,season][model]
                
                os.system(f'mkdir -p {region}_{season}_{model}/')
                os.system(f'cp src/* {region}_{season}_{model}/')
                
                os.chdir(f'{region}_{season}_{model}/')
                print(os.listdir())
                
                cmdprefix = 'cat ISORange.dat | sed -i '
                cmdT = f"-e 's/T(K):       0.0/T(K):       {T_tmp}/' "
                cmdRH = f"-e 's/RH(-):      0.0/RH(-):      {RH_tmp}/' "
                cmdNH3 = f"-e 's/TSO4:       0.0e-9/TSO4:       {SO4_tmp}/' "
                cmdoutput = f"-e 's/Output:     NW_DJF_nonuc_ISOOutput.nc/Output:     ISOOutput_{region}_{season}_{model}_constant_SO4.nc/' "
                cmdsuffix = 'ISORange.dat'
                
                cmd = cmdprefix+cmdT+cmdRH+cmdNH3+cmdoutput+cmdsuffix
                
                os.system(cmd)
                
                os.chdir('../')

os.chdir('../grid_model/ego_nonuclear_project/')

['CREDITS', 'how_to.txt', 'ISORange.dat', 'ISORange_original.dat', 'ISORange_test.dat', 'isorropiaII_main_mod.F', 'isorropiaii_main_mod.mod', 'isorropiaII_main_mod.o', 'Makefile', 'multi_isorropiaII.F90', 'multi_isorropiaII.o', 'runISORROPIA']
['CREDITS', 'how_to.txt', 'ISORange.dat', 'ISORange_original.dat', 'ISORange_test.dat', 'isorropiaII_main_mod.F', 'isorropiaii_main_mod.mod', 'isorropiaII_main_mod.o', 'Makefile', 'multi_isorropiaII.F90', 'multi_isorropiaII.o', 'runISORROPIA']
['CREDITS', 'how_to.txt', 'ISORange.dat', 'ISORange_original.dat', 'ISORange_test.dat', 'isorropiaII_main_mod.F', 'isorropiaii_main_mod.mod', 'isorropiaII_main_mod.o', 'Makefile', 'multi_isorropiaII.F90', 'multi_isorropiaII.o', 'runISORROPIA']
['CREDITS', 'how_to.txt', 'ISORange.dat', 'ISORange_original.dat', 'ISORange_test.dat', 'isorropiaII_main_mod.F', 'isorropiaii_main_mod.mod', 'isorropiaII_main_mod.o', 'Makefile', 'multi_isorropiaII.F90', 'multi_isorropiaII.o', 'runISORROPIA']
['CREDITS', 'how_to.txt'

## Save our dataframe

In [173]:
#### save our data out ####
isorropia_df.to_csv('./data/ISORROPIA_data.csv')